In [1]:
import matplotlib
import os
import time
import random
import numpy as np
#from torchviz import make_dot

In [2]:
!pwd

/home/otalora/miccai22/fedem


In [3]:
from network import *
from framework import FedRod
from preprocessing import dataPreprocessing
##Networks
import monai

cuda


In [4]:
lieu = 'SCAN' #SCAN or LAPTOP  or SERVER
isles_data_root = ''
if lieu == 'SCAN':
    isles_data_root = '/str/data/ASAP/miccai22_data/isles/federated/'
if lieu == 'LAPTOP':
    isles_data_root = '/Users/sebastianotalora/work/postdoc/data/ISLES/federated/'

exp_root = '/Users/sebastianotalora/work/tmi/fedem'
modality = 'Tmax'
batch_size = 2


In [5]:
clients=["center1", "center2", "center3", "center4"]
#from SCAFFOLD manuscript, global_learning_rate should be = sqrt(#Samples sites)
local_epoch, global_epoch = 1, 20
#no sampling
K=len(clients)

local_lr, global_lr = 0.00932, 1.7 #np.sqrt(K)

_, centers_data_loaders, all_test_loader, _ = dataPreprocessing(isles_data_root, modality, 4, 2)

#move center 3 at the end of the dataloaders
tmp = centers_data_loaders[2]
centers_data_loaders[2]=centers_data_loaders[3]
centers_data_loaders[3]=tmp

options = {'K': K, 'l_epoch': local_epoch, 'B': batch_size, 'g_epoch': global_epoch, 'clients': clients,
           'l_lr':local_lr, 'g_lr':global_lr, 'dataloader':centers_data_loaders, 'suffix': 'FedRod', 
           'scaffold_controls': False, 'fed_rod':True}

15 3 4
41 8 8
1 0 1
9 2 2


In [6]:
fed_rod = FedRod(options)

In [8]:
fed_rod.train_server(global_epoch=10, local_epoch=1, global_lr=global_lr, local_lr=local_lr)

*** global_epoch: 1 ***
center1 loss: nan
center2 loss: nan
center3 loss: nan
*** global_epoch: 2 ***
center1 loss: nan


KeyboardInterrupt: 

In [2]:
3e-4

0.0003

In [ ]:
fed_rod.nns

In [ ]:
fed_rod.test(all_test_loader)

In [ ]:
len(all_test_loader)

In [ ]:
print(len(centers_data_loaders[0][0]))
print(len(centers_data_loaders[0][1]))
print(len(centers_data_loaders[0][2]))